In [2]:
import numpy as np
import pandas as pd
from keras.utils import image_dataset_from_directory
import cv2

In [3]:
df_train="/kaggle/input/skin-disease-dataset/skin-disease-datasaet/train_set"
df_val="/kaggle/input/skin-disease-dataset/skin-disease-datasaet/test_set"

In [4]:
d=cv2.imread("/kaggle/input/skin-disease-dataset/skin-disease-datasaet/train_set/BA-impetigo/11_BA-impetigo (77).jpg")
d.shape

(212, 212, 3)

In [5]:
train_data=image_dataset_from_directory(
df_train,
seed=123,
shuffle=True,
image_size=(226,226),
)
val_data=image_dataset_from_directory(
df_val,
seed=123,
shuffle=True,
image_size=(226,226),
)

Found 924 files belonging to 8 classes.
Found 233 files belonging to 8 classes.


# # Deep learning

In [6]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,BatchNormalization,TimeDistributed,Input,Dropout
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [7]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1
)

In [27]:
inputs=Input(shape=(226,226,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)

M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
B=BatchNormalization()(M)
D=Dropout(0.5)(B)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)

M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

B=BatchNormalization()(M)
D=Dropout(0.5)(B)
C=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
C=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
B=BatchNormalization()(M)
F=Flatten()(B)

D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(F)
# D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
outputs=Dense(8,activation="softmax")(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [28]:
model.fit(train_data,epochs=50,validation_data=val_data,callbacks=[early_stopping])

Epoch 1/50


I0000 00:00:1726480461.613528     112 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_reduce_window_fusion_3', 8 bytes spill stores, 8 bytes spill loads



28/29 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.1883 - loss: 16.4244

I0000 00:00:1726480473.823547     112 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_reduce_window_fusion_3', 8 bytes spill stores, 8 bytes spill loads



29/29 ━━━━━━━━━━━━━━━━━━━━ 26s 516ms/step - accuracy: 0.1902 - loss: 16.3638 - val_accuracy: 0.1888 - val_loss: 22.0177
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 173ms/step - accuracy: 0.2560 - loss: 14.4504 - val_accuracy: 0.1588 - val_loss: 18.9807
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 173ms/step - accuracy: 0.4411 - loss: 12.9596 - val_accuracy: 0.2318 - val_loss: 13.5906
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 175ms/step - accuracy: 0.5072 - loss: 11.5495 - val_accuracy: 0.2876 - val_loss: 13.2208
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 178ms/step - accuracy: 0.6565 - loss: 10.2750 - val_accuracy: 0.3176 - val_loss: 11.5431
Epoch 5: early stopping


In [29]:
loss,accuracy=model.evaluate(val_data)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3044 - loss: 11.5185
